In [1]:
%pylab inline
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import pickle

Populating the interactive namespace from numpy and matplotlib


## Get the historical prime rates using BeautifulSoup

In [2]:
def scrape_prime():
    url = 'http://www.fedprimerate.com/wall_street_journal_prime_rate_history.htm'
    page = requests.get(url)
    soup=BeautifulSoup(page.content,"html")
    rows = soup.find_all(class_='p-1-b-c') #find all rows
    rlist = []
    for i in rows[4:520]:
         rlist.append(i.get_text())
    date = []
    rate = []
    for k,v in enumerate(rlist):
        if k % 2 == 0:
            rate.append(v)
        else:
            date.append(v)
    df_dict = {'date':date,
               'rate':rate}
    df = pd.DataFrame.from_dict(df_dict)
    return df

In [3]:
df = scrapeprime()

## Data Cleaning

In [3]:
def clean_result(df):
    """Takes in the scrapped data, cleans it and outputs a csv."""
    df.rate = df.rate.shift(-1) 
    df.drop(253, inplace=True)
    df.reset_index(inplace=True)
    df.drop(columns=['index'], inplace=True)
    df.date = df.date.str.replace('\n ', '', regex=False)
    df['date'] = pd.to_datetime(df['date'])
    df.iloc[256][1] = 5.25
    df.set_value(256, 'rate', 5.5)
    df.to_csv('clean_prime.csv')

In [ ]:
clean_result(df)